In [1]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import json
import random as rand
import datetime

In [2]:
with open('dummy_data/timelined_sentiments.json') as f:
  data = json.load(f)
  data =data['features']['2020-04-22']

In [3]:
real_data = pd.read_csv('output_sents_w_country9900.csv')
ids = real_data['id']

In [4]:
real_data.head()

,Unnamed: 0,Unnamed: 0.1,id,text,coordinates,geo,place,date,retweet_count,retweeted,prediction,log_pos_proba,pos_proba,country
0,0,0,1252824529472114699,2020 #UK #Jobs statistics (22-04-2020\r\r\ncom...,"{'type': 'Point', 'coordinates': [-0.77613903,...","{'type': 'Point', 'coordinates': [53.39363907,...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:00:18,0,False,0,-0.811593,0.444150,United Kingdom
1,1,1,1252824738985947141,Where was the last place you’ve visited and wh...,"{'type': 'Point', 'coordinates': [100.6065, 13...","{'type': 'Point', 'coordinates': [13.69455, 10...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:01:08,0,False,0,-1.027491,0.357904,ประเทศไทย
2,2,2,1252825364499324928,#rave #wearyourmask #protectyourface #coronavi...,"{'type': 'Point', 'coordinates': [-118.2445, 3...","{'type': 'Point', 'coordinates': [34.0564, -11...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:03:37,0,False,4,-0.237127,0.788891,United States of America
3,3,3,1252826143012487170,"As I sit and ponder of a world pre-Corona, I c...","{'type': 'Point', 'coordinates': [73.818, 15.4...","{'type': 'Point', 'coordinates': [15.493, 73.8...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:06:43,0,False,4,-0.164305,0.848483,India
4,4,4,1252826317218668550,"I stay at work for you, you stay at home for u...","{'type': 'Point', 'coordinates': [101.69958855...","{'type': 'Point', 'coordinates': [3.15270339, ...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:07:25,0,False,4,-0.607934,0.544474,Malaysia


In [5]:
real_data['prediction'].value_counts()

4    14107
0     4182
Name: prediction, dtype: int64

In [6]:
real_data['date'] = pd.to_datetime(real_data['date'])
real_data.head()

,Unnamed: 0,Unnamed: 0.1,id,text,coordinates,geo,place,date,retweet_count,retweeted,prediction,log_pos_proba,pos_proba,country
0,0,0,1252824529472114699,2020 #UK #Jobs statistics (22-04-2020\r\r\ncom...,"{'type': 'Point', 'coordinates': [-0.77613903,...","{'type': 'Point', 'coordinates': [53.39363907,...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:00:18,0,False,0,-0.811593,0.444150,United Kingdom
1,1,1,1252824738985947141,Where was the last place you’ve visited and wh...,"{'type': 'Point', 'coordinates': [100.6065, 13...","{'type': 'Point', 'coordinates': [13.69455, 10...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:01:08,0,False,0,-1.027491,0.357904,ประเทศไทย
2,2,2,1252825364499324928,#rave #wearyourmask #protectyourface #coronavi...,"{'type': 'Point', 'coordinates': [-118.2445, 3...","{'type': 'Point', 'coordinates': [34.0564, -11...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:03:37,0,False,4,-0.237127,0.788891,United States of America
3,3,3,1252826143012487170,"As I sit and ponder of a world pre-Corona, I c...","{'type': 'Point', 'coordinates': [73.818, 15.4...","{'type': 'Point', 'coordinates': [15.493, 73.8...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:06:43,0,False,4,-0.164305,0.848483,India
4,4,4,1252826317218668550,"I stay at work for you, you stay at home for u...","{'type': 'Point', 'coordinates': [101.69958855...","{'type': 'Point', 'coordinates': [3.15270339, ...",Place(_api=<tweepy.api.API object at 0x0000018...,2020-04-22 05:07:25,0,False,4,-0.607934,0.544474,Malaysia


In [7]:
states = {}
for d in data:
    states[d['properties']['state']] = {}
    states[d['properties']['state']]['geometry'] = d['geometry']
    states[d['properties']['state']]['messages'] = []

In [8]:
# random selection for messages to draw from
import re
import numpy as np

state_names = list(states.keys())
chance_of_reply = 0.75
tweet_rate = 0.95
messages = []
ident = []

for index in range(len(real_data)):
    new_message = {}
    new_message['state'] = rand.choice(state_names)
    new_message['hashtags'] = re.findall(r"#(\w+)", real_data.loc[index, "text"])
    new_message['message'] = real_data.loc[index, "text"]
    for i in range(len(new_message['hashtags'])):
        new_message['hashtags'][i] = '#' + new_message['hashtags'][i]
    new_message['sentiment'] = real_data.loc[index, "prediction"]
    if new_message['sentiment'] == 4:
        new_message['sentiment'] = np.random.normal(0.6, 0.15)
    if new_message['sentiment'] == 0:
        new_message['sentiment'] = np.random.normal(0.4, 0.15)
    
    if rand.uniform(0, 1) < chance_of_reply and index is not 0:
        new_message['isReply'] = True
        new_message['replyTo'] = rand.choice(ident)
    else:
        new_message['isReply'] = False
        new_message['replyTo'] = 0
    new_message['id'] = str(real_data.loc[index, "id"])
    new_message["date"] = real_data.loc[index, "date"]
    ident.append(new_message['id'])
    
    messages.append(new_message)

<>:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-8-8c838f803e0f>:24: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if rand.uniform(0, 1) < chance_of_reply and index is not 0:


In [9]:
messages = messages[0:5000]

In [10]:
for message in messages:
    states[message['state']]['messages'].append(message)

In [11]:
print(states['New York']['messages'][0: 5])

[{'state': 'New York', 'hashtags': ['#coronavirus', '#exmoordistillery', '#shoplocal', '#staysafe', '#withypool'], 'message': 'New Exmoor Distillery HAND SANTITIZER in stock! #coronavirus #exmoordistillery #shoplocal #staysafe #withypool… https://t.co/GwY73W0zM6', 'sentiment': 0.6875799536098616, 'isReply': True, 'replyTo': '1252839417711964162', 'id': '1252876703015161858', 'date': Timestamp('2020-04-22 08:27:37')}, {'state': 'New York', 'hashtags': ['#coronavirus', '#socialisolation'], 'message': 'The #coronavirus #socialisolation death toll of physically healthy young people is really starting to mount. The la… https://t.co/jw6vJGFR1v', 'sentiment': 0.2617972613367152, 'isReply': True, 'replyTo': '1252827987210170373', 'id': '1252892507308003335', 'date': Timestamp('2020-04-22 09:30:26')}, {'state': 'New York', 'hashtags': [], 'message': 'Yesterday, SafeHouse served as a COVID-19 testing site for staff, volunteers, and any walk-ups. 201 were tested and… https://t.co/fpYANDKg09', 'se

In [12]:
output = {}
output['type'] = "FeatureCollection"
output['features'] = states

In [13]:
date_handler = lambda obj: (
    obj.isoformat()
    if isinstance(obj, (datetime.datetime, datetime.date))
    else None
)

with open('dummy_data/real.json', 'w') as outfile:
    json.dump(output, outfile, default=date_handler)